In [1]:
test_input1 = """.....
.S-7.
.|.|.
.L-J.
....."""

test_input2 = """-L|F7
7S-7|
L|7||
-L-J|
L|-JF"""

test_input3 = """..F7.
.FJ|.
SJ.L7
|F--J
LJ..."""

test_input4 = """...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
..........."""

test_input5 = """.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ..."""


test_input6 = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

In [2]:
import numpy as np
import itertools

def parse_board(input):
    return list(map(list, input.splitlines()))

def print_board(board):
    lines = [''.join(board[i]) for i in range(board.shape[0])]

    for l in lines:
        print(l)

In [3]:
# (5, -3) means 5 down 3 left

# we're going to store the two ends for each pipe
pipe_openings = {
    "F": ((1, 0), (0, 1)),
    "-": ((0, -1), (0, 1)),
    "7": ((0, -1), (1, 0)),
    "|": ((-1, 0), (1, 0)),
    "J": ((-1, 0), (0, -1)),
    "L": ((0, 1), (-1, 0))
}

In [4]:
input = open("inputs/10").read()
board = np.array(parse_board(input))

# board = np.array(parse_board(test_input2))
# board = np.array(parse_board(test_input3))
# board = np.array(parse_board(test_input4))
# board = np.array(parse_board(test_input5))
# board = np.array(parse_board(test_input6))

In [5]:
start_index = np.concatenate(np.where(board == 'S'))

def in_bounds(board, i, j):
    return (i >= 0) and (i < board.shape[0]) and (j >= 0) and (j < board.shape[1])

def try_walk(test_board, start_index):
    current_position = start_index.copy()
    enter_dir = None

    positions = []

    for i in itertools.count():
        # print(i)

        if not in_bounds(test_board, *current_position):
            # Walked into invalid position
            return None

        current_pipe = test_board[*current_position]

        if current_pipe == '.':
            # Floor
            return None

        opening_dirs = pipe_openings[current_pipe]

        if enter_dir:
            # use this to infer what direction we're moving
            # i.e., choose the one which doesn't return us from whence we came
            # assuming such a direction exists! if not, this is an invalid location
            # this checks "pipe compatibility"
            backwards_dir = next((i for i, t in enumerate(opening_dirs) if t == (-enter_dir[0], -enter_dir[1])), None)

            if backwards_dir is None:
                # Invalid location. No way to retreat
                return None
            else:
                enter_dir = opening_dirs[(backwards_dir + 1) % 2]
        else:
            # we're starting. just pick a direction
            enter_dir = opening_dirs[0]

        # if we reached this point the cell is valid
        # do whatever computation and then proceed in the remaining direction

        # print(current_pipe)

        if (i > 0) and (tuple(current_position) == tuple(start_index)):
            # We've arrived back to the start :)
            break

        positions.append(current_position.copy())
        current_position += enter_dir
    
    return positions, i // 2

for pipe_option in pipe_openings.keys():
    # print(pipe_option)
    test_board = board.copy()
    test_board[*start_index] = pipe_option

    walk_result = try_walk(test_board, start_index)

    if walk_result is not None:
        main_loop_positions, max_d = walk_result
        break
else:
    assert False

    # print(max_d)
    # print()

In [6]:
# now that we know what the pipe type of S is
board = test_board

# furthermore, for the purpose of pt2 we can set everything not in the main loop to be floor


In [7]:
mask = np.ones_like(board, dtype=bool)

In [8]:
main_loop_positions[:5]

[array([31, 28]),
 array([31, 27]),
 array([32, 27]),
 array([33, 27]),
 array([33, 28])]

In [9]:
board.shape

(140, 140)

In [10]:
stacked_main_loop = np.vstack(main_loop_positions)
mask[stacked_main_loop[:, 0], stacked_main_loop[:, 1]] = False

mask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [11]:
board[mask] = '.'

In [12]:
print_board(board)

............................................................................................................................................
............................................................................................................................................
............................................................................................................................................
............................................................F7..............................................................................
.......................................................F7...||..............................................................................
.................................................F7....|L7..||......F7............................F7........................................
....................................F7...........|L7F-7L7|.FJL7F7...||..............F7...........FJ|F7......................................
.............

In [13]:
enlarged_board = np.full((board.shape[0]*2, board.shape[1]*2), '*', dtype=board.dtype)

In [14]:
enlarged_board[::2, ::2] = board

In [15]:
print_board(enlarged_board)

.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
****************************************************************************************************************************************************************************************************************************************************************************************
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
*************************************************************************************************************************************************************

In [16]:
# Now we need to just "extend" the pipes with some "blocker" character: 'x'

pipe_indices = np.where((enlarged_board != '*') & (enlarged_board != '.'))

In [17]:
for pipe_location in zip(*pipe_indices):
    # we put blockers in the direction of the pipe openings
    pipe_location = np.array(pipe_location)

    opening_dirs = pipe_openings[enlarged_board[*pipe_location]]

    blocker1_location = pipe_location + np.array(opening_dirs[0])
    blocker2_location = pipe_location + np.array(opening_dirs[1])

    enlarged_board[*blocker1_location] = 'x'
    enlarged_board[*blocker2_location] = 'x'

In [18]:
print_board(enlarged_board)

.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
****************************************************************************************************************************************************************************************************************************************************************************************
.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*
*************************************************************************************************************************************************************

In [19]:
# finally, we need to find the connected components
floor_indices = set(zip(*np.where(enlarged_board == '.')))

In [20]:
# walk_directions = [d for d in itertools.product((-1, 0, 1), (-1, 0, 1)) if not (d[0] == 0 and d[1] == 0)]
walk_directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

In [21]:
import sys
sys.setrecursionlimit(100_000)

trapped_count = 0

for i in itertools.count():
    if len(floor_indices) == 0:
        break

    test_start = list(floor_indices)[0]

    seen = set()
    reached_edge = False

    def walk(position):
        seen.add(position)

        global reached_edge

        if not in_bounds(enlarged_board, *position):
            reached_edge = True
            return

        c = enlarged_board[position]

        if (c != '.') and (c != '*'):
            # blocked
            return

        for dir in walk_directions:
            next_position = tuple(np.array(position) + np.array(dir))

            if next_position not in seen:
                walk(next_position)

    walk(test_start)

    len(seen & floor_indices)

    if not reached_edge:
        seen_floor = seen & floor_indices
        print(seen_floor)
        trapped_count += len(seen_floor)
    
    floor_indices -= seen

{(126, 126), (136, 158), (130, 124), (118, 122), (128, 154), (148, 132), (140, 128), (140, 146), (132, 142), (132, 160), (152, 138), (144, 134), (208, 146), (144, 152), (196, 190), (208, 164), (158, 154), (136, 144), (128, 140), (128, 158), (148, 136), (140, 132), (132, 128), (152, 106), (148, 154), (140, 150), (132, 146), (162, 156), (144, 138), (124, 156), (126, 98), (136, 130), (118, 112), (136, 148), (128, 144), (128, 162), (152, 92), (140, 136), (132, 132), (120, 176), (132, 150), (78, 188), (146, 152), (122, 114), (124, 142), (124, 160), (136, 134), (128, 130), (136, 152), (128, 148), (120, 144), (86, 142), (150, 154), (120, 162), (140, 140), (132, 136), (146, 156), (84, 92), (106, 110), (124, 146), (158, 148), (112, 190), (136, 120), (200, 132), (136, 138), (128, 134), (128, 152), (120, 148), (144, 78), (140, 126), (192, 164), (132, 122), (142, 154), (134, 150), (130, 198), (184, 178), (172, 176), (88, 192), (84, 78), (194, 200), (146, 160), (178, 112), (124, 150), (158, 152), (

In [22]:
enlarged_board.shape

(280, 280)

In [23]:
trapped_count

443